In [1]:
# Importing libraries
!pip install git+https://github.com/tensorflow/docs
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np
import imageio
from IPython import display
from urllib import request
import re
import pprint
import glob
import pandas as pd
from IPython.display import Video

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-mnyzqmxu
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-mnyzqmxu
  Resolved https://github.com/tensorflow/docs to commit 476e61b24de218a85fbd71edffdd314c3c6a8b61
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=184468 sha256=095cbf701b41d1e8ca71923b05a65a7bb19479b24a48fe24a28d55c67ae6c712
  Stored in directory: /tmp/pip-ephem-wheel-cache-uu2od3k_/wheels/3b/ee/a2/ab4d36a9a4af495bcb936f3e849d4b497b65fa40548a68d6c3
Successfully built tensorflow-docs


In [2]:
# Mounting google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-600/1").signatures['default']

In [4]:
# Get the kinetics-600 action labels from the GitHub repository.

with open('/content/drive/MyDrive/Scomp/Visual/Dataset/labels.txt') as f:
    l = f.readlines()
    labels=len(l)

with open('/content/drive/MyDrive/Scomp/Visual/Dataset/labels.txt','r') as f_in:
    lines = f_in.read()
    listed=re.split('; |, |\*|\n',lines)
    pprint.pprint(listed)

FileNotFoundError: ignored

In [ ]:
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)

      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0
def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=25)
  return embed.embed_file('./animation.gif')

In [ ]:
# HAPPY

path = "/content/drive/MyDrive/Scomp/Visual/Dataset/Youtube Data/Happy"
wav_files = glob.glob(path + "/*.mp4")
for i in wav_files:
    
    video_path= i
    sample_video = load_video(video_path)[:100]
    sample_video.shape

    def predict(sample_video):
      # Add a batch axis to the sample video.
      model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

      logits = i3d(model_input)['default'][0]
      probabilities = tf.nn.softmax(logits)

      print("Top 2 actions:")
      for j in np.argsort(probabilities)[::-1][:2]:
        print(f"  {listed[j]:22}: {probabilities[j] * 100:2.2f}%")
    predict(sample_video)

Top 2 actions:
  answering questions   : 27.28%
  raising eyebrows      : 10.30%
Top 2 actions:
  lifting hat           : 38.13%
  tossing coin          : 6.66%
Top 2 actions:
  playing squash or racquetball: 13.25%
  smoking               : 8.20%
Top 2 actions:
  adjusting glasses     : 8.86%
  photobombing          : 8.66%
Top 2 actions:
  answering questions   : 15.25%
  smoking               : 10.08%
Top 2 actions:
  answering questions   : 22.05%
  chewing gum           : 15.33%
Top 2 actions:
  raising eyebrows      : 60.21%
  shaving head          : 7.93%
Top 2 actions:
  trimming shrubs       : 86.46%
  weaving basket        : 7.78%
Top 2 actions:
  pumping gas           : 38.35%
  getting a tattoo      : 15.58%
Top 2 actions:
  applying cream        : 24.44%
  using inhaler         : 11.99%


In [ ]:
# SAD
path = "/content/drive/MyDrive/Scomp/Visual/Dataset/Youtube Data/Sad"
wav_files = glob.glob(path + "/*.mp4")
for i in wav_files:
    video_path= i
    sample_video = load_video(video_path)[:100]
    sample_video.shape

    def predict(sample_video):
      # Add a batch axis to the sample video.
      model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

      logits = i3d(model_input)['default'][0]
      probabilities = tf.nn.softmax(logits)

      print("Top 2 actions:")
      for i in np.argsort(probabilities)[::-1][:2]:
        print(f"  {listed[i]:22}: {probabilities[i] * 100:2.2f}%")
   
    predict(sample_video)

Top 2 actions:
  applying cream        : 42.16%
  scrubbing face        : 11.19%
Top 2 actions:
  using inhaler         : 21.34%
  smoking               : 14.39%
Top 2 actions:
  texting               : 32.69%
  looking at phone      : 18.64%
Top 2 actions:
  answering questions   : 20.37%
  attending conference  : 6.47%
Top 2 actions:
  using inhaler         : 35.05%
  applying cream        : 9.93%
Top 2 actions:
  putting on foundation : 19.56%
  scrubbing face        : 16.08%
Top 2 actions:
  playing ukulele       : 28.76%
  using inhaler         : 21.01%
Top 2 actions:
  crying                : 17.88%
  using inhaler         : 17.74%
Top 2 actions:
  using inhaler         : 6.83%
  answering questions   : 5.04%
Top 2 actions:
  giving or receiving award: 27.57%
  sneezing              : 8.92%


In [ ]:
# SURPRISE

path = "/content/drive/MyDrive/Scomp/Visual/Dataset/Youtube Data/Surprise"
wav_files = glob.glob(path + "/*.mp4")
for i in wav_files:

    video_path= i
    sample_video = load_video(video_path)[:100]
    sample_video.shape

    def predict(sample_video):
      # Add a batch axis to the sample video.
      model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

      logits = i3d(model_input)['default'][0]
      probabilities = tf.nn.softmax(logits)

      print("Top 2 actions:")
      for i in np.argsort(probabilities)[::-1][:2]:
        print(f"  {listed[i]:22}: {probabilities[i] * 100:2.2f}%")
    predict(sample_video)

Top 2 actions:
  changing gear in car  : 18.59%
  texting               : 7.79%
Top 2 actions:
  setting table         : 84.52%
  scrapbooking          : 4.01%
Top 2 actions:
  moving furniture      : 31.11%
  laying tiles          : 21.99%
Top 2 actions:
  tie dying             : 14.95%
  building cabinet      : 7.98%
Top 2 actions:
  hugging (not baby)    : 17.36%
  finger snapping       : 13.14%
Top 2 actions:
  opening door          : 77.03%
  opening refrigerator  : 4.60%
Top 2 actions:
  reading newspaper     : 29.48%
  delivering mail       : 21.73%
Top 2 actions:
  opening door          : 56.91%
  opening refrigerator  : 19.56%
Top 2 actions:
  skipping rope         : 32.73%
  photobombing          : 18.08%
Top 2 actions:
  cleaning windows      : 25.84%
  sanding floor         : 5.61%


In [ ]:
# ANGER

path = "/content/drive/MyDrive/Scomp/Visual/Dataset/Youtube Data/Anger"
wav_files = glob.glob(path + "/*.mp4")
for i in wav_files:
    file_name = os.path.basename(i)
    pth=(os.path.splitext(file_name)[0])
    video_path= i
    sample_video = load_video(video_path)[:100]
    sample_video.shape

    def predict(sample_video):
      # Add a batch axis to the sample video.
      model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

      logits = i3d(model_input)['default'][0]
      probabilities = tf.nn.softmax(logits)

      print("Top 2 actions:")
      for i in np.argsort(probabilities)[::-1][:2]:
        print(f"  {listed[i]:22}: {probabilities[i] * 100:2.2f}%")
    predict(sample_video)

Top 2 actions:
  laughing              : 48.70%
  sneezing              : 11.81%
Top 2 actions:
  headbanging           : 60.50%
  air drumming          : 16.70%
Top 2 actions:
  skydiving             : 27.49%
  carving ice           : 14.45%
Top 2 actions:
  using bagging machine : 20.48%
  grinding meat         : 7.17%
Top 2 actions:
  arguing               : 82.51%
  shaking hands         : 2.31%
Top 2 actions:
  using segway          : 65.56%
  motorcycling          : 12.95%
Top 2 actions:
  attending conference  : 28.59%
  answering questions   : 12.85%
Top 2 actions:
  talking on cell phone : 15.58%
  texting               : 9.12%
Top 2 actions:
  talking on cell phone : 10.41%
  directing traffic     : 10.21%
Top 2 actions:
  using inhaler         : 41.18%
  trimming or shaving beard: 7.39%


In [ ]:
# NEUTRAL

path = "/content/drive/MyDrive/Scomp/Visual/Dataset/Youtube Data/Neutral"
wav_files = glob.glob(path + "/*.mp4")
for i in wav_files:
    video_path= i
    sample_video = load_video(video_path)[:100]
    sample_video.shape

    def predict(sample_video):
      # Add a batch axis to the sample video.
      model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

      logits = i3d(model_input)['default'][0]
      probabilities = tf.nn.softmax(logits)

      print("Top 2 actions:")
      for i in np.argsort(probabilities)[::-1][:2]:
        print(f"  {listed[i]:22}: {probabilities[i] * 100:2.2f}%")

    predict(sample_video)

Top 2 actions:
  using inhaler         : 28.90%
  presenting weather forecast: 10.33%
Top 2 actions:
  answering questions   : 19.38%
  sign language interpreting: 6.48%
Top 2 actions:
  massaging neck        : 24.13%
  trimming or shaving beard: 20.32%
Top 2 actions:
  answering questions   : 12.62%
  chewing gum           : 9.69%
Top 2 actions:
  answering questions   : 20.33%
  opening refrigerator  : 7.76%
Top 2 actions:
  presenting weather forecast: 67.45%
  making balloon shapes : 2.23%
Top 2 actions:
  trimming or shaving beard: 7.94%
  waving hand           : 6.50%
Top 2 actions:
  answering questions   : 13.64%
  using inhaler         : 11.03%
Top 2 actions:
  presenting weather forecast: 33.01%
  giving or receiving award: 8.15%
Top 2 actions:
  presenting weather forecast: 59.35%
  giving or receiving award: 21.86%
